In [8]:
%matplotlib notebook

import numpy as np
import os

from gf321_utils import marker_utils
from gf321_utils import bvh_utils
from gf321_utils import plot_utils

In [9]:
#Import data
data_filepath = 'neural_lbs_data/'
joints = np.load(data_filepath + 'all_joints_aug.npy')
markers = np.load(data_filepath + 'all_markers_aug.npy')

In [10]:
#Make local stuff
root_tfs = bvh_utils.create_local_tfs_root(joints)
markers_local = marker_utils.make_markers_local(markers,root_tfs)
joints_local = bvh_utils.make_joints_local(joints)

In [ ]:
def corrupt(markers, occlude_std, shift_std, max_shift):
    
    frame_num = markers.shape[0]
    
    #Sample probabilities at which to occlude/shift
    occlude_prob = np.random.normal(0, occlude_std, frame_num)
    shift_prob = np.random.normal(0, shift_std, frame_num)
    
    #Sample using clipped probabilities
    occlusion = np.random.binomial(1, min(occlude_prob, 2*occlude_std), frame_num)
    shift = np.random.binomial(1, min(shift_prob, 2*shift_std), frame_num)
    
    #Sample the magnitude by which to shift each marker
    shift_mag = np.random.uniform(-max_shift, max_shift, frame_num)
    
    #Move shifted markers and place occluded ones at zero
    